In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from regpy.operators.convolution import GaussianBlur
from regpy.vecsps import UniformGridFcts
from regpy.solvers import TikhonovRegularizationSetting, RegularizationSetting
from regpy.solvers.linear.tikhonov import TikhonovCG
from regpy.solvers.linear.primal_dual import PDHG
from regpy.solvers.linear.proximal_gradient import FISTA
import regpy.functionals as fct
from regpy.hilbert import L2
from regpy.stoprules import DualityGapStopping
from comparison_plot import comparison_plot
import logging

# Generate a test image as exact solution

In [ ]:
grid = UniformGridFcts((-1, 1, 256), (-1.5, 1, 256),dtype = float, periodic = True)
"""Space of real-valued functions on a uniform grid with rectangular pixels"""
X = grid.coords[0]; Y = grid.coords[1]
"""x and y coordinates."""
cross = 1.0*np.logical_or((abs(X)<0.01) * (abs(Y)<0.3),(abs(X)<0.3) * (abs(Y)<0.01)) 
rad = np.sqrt(X**2 + Y**2)
ring = 1.0*np.logical_and(rad>=0.9, rad<=0.925)
smallbox = (abs(X+0.55)<=0.05) * (abs(Y-0.55)<=0.05)
bubbles = (1.001+np.sin(50/(X+1.3)))*np.exp(-((Y+1.25)/0.1)**2)*(X>-0.8)*(X<0.8)

ramp = Y<=-1

objects = 50*(ring + 2.0*cross + 1.5*smallbox + 2*ramp -bubbles)
exact_sol = objects 


# Generate synthetic data with impulsive noise

In [ ]:
a=0.05
conv =  GaussianBlur(grid,a,pad_amount=((16,16),(16,16)))
"""Convolution operator \(f\mapsto h*f\) for the convolution kernel \(h(x)=\exp(-|x|_2^2/a^2)\)."""
blur = conv(exact_sol)
blur[blur<0] = 0.
data = blur.copy()
n,m=grid.shape
for j in range(64**2):
    nn = np.random.randint(n)
    mm = np.random.randint(m)
    data[nn,mm] =  np.random.randint(2000)-1000

comparison_plot(grid,exact_sol,data,title_left='noisy measurement data')

# Reconstruction with a quadratic data fidelity term
... fails completely for this data!

In [ ]:
alpha = 1e-4
setting = RegularizationSetting(conv,L2,L2)
solver = TikhonovCG(setting=setting, data = data, regpar = alpha,reltolx=1e-6)
fal,_ = solver.run()

comparison_plot(grid,exact_sol,fal,title_left='quadratic Tikhonov regularization')

# setting with Huber data fidelity and quadratic penalty term
We now set up a generalized Tikhonov regularization setting
$$
\hat{f} \in \mathrm{argmin}_f\left[\frac{1}{\alpha}H_{\sigma}(Tf-g^{\mathrm{obs}})+\|f\|^2\right]    
$$
where the Huber data fidelity term $H_{\sigma}$ is quadratic on coordinates in $[-\sigma,\sigma]$ and $|\cdot|-\sigma/2$ for large coordinates.

We test three solvers for this setting:
- FISTA applied to the primal problem
- FISTA applied to the dual problem
- Primal-Dual Hybrid Gradient (Chambolle-Pock) method

In [ ]:
sigma = 0.1
Sdat = (1./sigma)*fct.Huber(grid,sigma=sigma,eps =1e-10)
huber_setting = TikhonovRegularizationSetting(conv,L2,Sdat,alpha,data_fid_shift=data)
dual_setting = huber_setting.dualSetting()
threshold = 1e-1
max_iter=500

### FISTA applied to primal problem

In [ ]:
solverFISTA_primal = FISTA(huber_setting,logging_level=logging.DEBUG)
stop_FISTA_primal=DualityGapStopping(solverFISTA_primal,threshold = threshold, max_iter=max_iter,logging_level=logging.INFO)
f_FISTA_primal, _= solverFISTA_primal.run(stoprule=stop_FISTA_primal)

comparison_plot(grid,exact_sol,f_FISTA_primal,title_left='FISTA primal')

### FISTA applied to the dual problem

In [ ]:
solverFISTA_dual = FISTA(dual_setting,logging_level=logging.DEBUG)
stop_FISTA_dual=DualityGapStopping(solverFISTA_dual,threshold = threshold, max_iter=max_iter,logging_level=logging.INFO)
pFISTA_dual, Tstar_pFISTA_dual= solverFISTA_dual.run(stoprule=stop_FISTA_dual)

f_FISTA_dual = huber_setting.dualToPrimal(Tstar_pFISTA_dual,argumentIsOperatorImage=True)
comparison_plot(grid,exact_sol,f_FISTA_dual,title_left='FISTA')

### PDHG


In [ ]:
solverPDHG = PDHG(huber_setting,logging_level=logging.INFO)
stop_PDHG=DualityGapStopping(solverPDHG,threshold = threshold, max_iter=max_iter,logging_level=logging.INFO)
f_PDHG, _= solverPDHG.run(stoprule=stop_PDHG)

comparison_plot(grid,exact_sol,f_PDHG,title_left='PDHG')

### comparison of convergence rates of minimization methods

In [ ]:
plt.semilogy(stop_FISTA_primal.gap_stat,label='FISTA primal')
plt.semilogy(stop_FISTA_dual.gap_stat,label='FISTA dual')
plt.semilogy(stop_PDHG.gap_stat,label='PDHG')
plt.legend()
plt.xlabel('it. step'); plt.ylabel('duality gap')
plt.title('Huber data fid., quadratic penalty')

# setting with Huber data fidelity and quadratic penalty term with bilateral constraints
We now set up a generalized Tikhonov regularization setting
$$
\hat{f} \in \mathrm{argmin}_{0\leq f\leq ub}\left[\frac{1}{\alpha}H_{\sigma}(Tf-g^{\mathrm{obs}})+\|f\|_{L^2}^2\right]    
$$
where the Huber data fidelity term $H_{\sigma}$ is quadratic on coordinates in $[-\sigma,\sigma]$ and $|\cdot|-\sigma/2$ for large coordinates.

We test three solvers for this setting:
- FISTA applied to the primal problem
- FISTA applied to the dual problem
- Primal-Dual Hybrid Gradient (Chambolle-Pock) method

In [ ]:
sigma = 0.1
Sdat = (1./sigma)*fct.Huber(grid,sigma=sigma,eps=1e-10)
Rfun = fct.QuadraticBilateralConstraints(grid,lb=0,ub=100,eps=1e-10)
huber_constr_setting = TikhonovRegularizationSetting(conv,Rfun,Sdat,alpha,data_fid_shift=data,logging_level=logging.DEBUG)
dual_constr_setting = huber_constr_setting.dualSetting()
threshold =0.1
max_iter = 500

### FISTA applied to primal setting 

In [ ]:
solverFISTA_constr_primal = FISTA(huber_constr_setting,logging_level=logging.DEBUG)
stop_FISTA_constr_primal=DualityGapStopping(solverFISTA_constr_primal,threshold = threshold, max_iter=max_iter,logging_level=logging.DEBUG)
f_FISTA_constr_primal, _= solverFISTA_constr_primal.run(stoprule=stop_FISTA_constr_primal)

comparison_plot(grid,exact_sol,f_FISTA_constr_primal,title_left='FISTA constraint primal')

### FISTA applied to the dual setting

In [ ]:
solverFISTA_constr_dual = FISTA(dual_constr_setting,logging_level=logging.DEBUG)
stop_FISTA_constr_dual=DualityGapStopping(solverFISTA_constr_dual,threshold = threshold, max_iter=max_iter,logging_level=logging.INFO)
pFISTA_constr_dual, Tstar_pFISTA_constr_dual= solverFISTA_constr_dual.run(stoprule=stop_FISTA_constr_dual)

f_FISTA_constr_dual = huber_constr_setting.dualToPrimal(Tstar_pFISTA_constr_dual,argumentIsOperatorImage=True)
comparison_plot(grid,exact_sol,f_FISTA_constr_dual,title_left='FISTA dual')

### PDHG

In [ ]:
solverPDHG_constr = PDHG(huber_constr_setting,logging_level=logging.INFO)
stop_PDHG_constr=DualityGapStopping(solverPDHG_constr,threshold = threshold, max_iter=max_iter,logging_level=logging.INFO)
f_PDHG_constr, _= solverPDHG_constr.run(stoprule=stop_PDHG_constr)

comparison_plot(grid,exact_sol,f_PDHG_constr,title_left='PDHG')

### comparison of convergence rates of minimization methods

In [ ]:
plt.semilogy(stop_FISTA_constr_primal.gap_stat,label='FISTA primal')
plt.semilogy(stop_FISTA_constr_dual.gap_stat,label='FISTA dual')
plt.semilogy(stop_PDHG_constr.gap_stat,label='PDHG')
plt.legend()
plt.xlabel('it. step'); plt.ylabel('duality gap')
plt.title('Huber data fid., quadratic constr. penalty')

# setting with \(L^1\) fidelity and quadratic penalty term with bilateral constraints
We now set up a generalized Tikhonov regularization setting
$$
\hat{f} \in \mathrm{argmin}_{0\leq f\leq ub}\left[\frac{1}{\alpha}\|Tf-g^{\mathrm{obs}}\|_{L^1}+\|f\|_{L^2}^2.\right]    
$$

We test two solvers for this setting:
- FISTA applied to the dual problem
- Primal-Dual Hybrid Gradient (Chambolle-Pock) method

In [ ]:

Sdat = fct.L1(grid)
Rfun = fct.QuadraticBilateralConstraints(grid,lb=0,ub=100,eps=1e-10)
huber_constr_setting = TikhonovRegularizationSetting(conv,Rfun,Sdat,alpha,data_fid_shift=data,logging_level=logging.DEBUG)
dual_constr_setting = huber_constr_setting.dualSetting()
threshold =0.1
max_iter = 500